# NLP and Data Extraction Challenge
## Data Science and Machine Learning

Internship Brazil Labs - 2024

> The idea of this test is to understand your way of thinking, coding, and problem-solving skills. There is no right answer, just follow your instincts and share with us the solutions of the challenges below.

# 1) Exploratory Data Analysis (EDA) and Visualization

>EDA is a set of process of examining, cleaning, and transforming data to uncover patterns, spot anomalies, test hypotheses, or check assumptions.


>Data visualization is the act of taking information (data) and placing it into a visual context, such as a map or graph. Data visualizations make big and small data easier for the human brain to understand, and visualization also makes it easier to detect patterns, trends, and outliers in groups of data.

The **first challenge** is: explore information from **dataset.csv** file.
<br>
You can choose your preferred techniques or tools, be aware that sometimes less is more, present meaningful insights, and show us what you have found interesting on this dataset!

### Solution

In [3]:
# open csv file

import csv
import pandas as pd

dados = pd.read_csv('resume_dataset.csv', delimiter=',', encoding='utf-8')

dados.shape

(962, 2)

### Explain the rationale of your decision about techniques and visualizations

# 2) NLP techniques / Machine Learning approach

The **second challenge** is: explore dataset information and apply NLP techniques or Machine Learning models to extract even more usefull information from data. Feel free to explore different machine learning models (e.g classification, clustering, etc) and make sure to explain the model results for each task you apply these ML models.
<br><br>
You can choose your preferred techniques or tools, be aware that sometimes less is more, present meaningful insights, and show us what you have found interesting on those files! (i.e: topic modeling, word distributions, etc)

### Machine Learning approach

### Explain the rationale of your NLP techniques/ML models usage and conclusions

# 3) Presentation

The **third** and last exercise is: present your findings to us, challenges of this exercise and how you handled them, and any other important information you will like to share with us.
<br><br>
We will be evaluating your Python code, how you organize your toughts and explanations during the presentation, and other aspects we see as important for a good presentation. You don't need to generate any output for this third task. 

Good luck! :)